# 05_html_to_jasper_snippets.ipynb


## Aufgabe

In diesem Notebook sollen HTML-Elemente in JasperReport-XML-Bausteine konvertiert werden.

Ziel: Umwandlung von HTML-Elementen in JasperReport-kompatible XML-Snippets, die in JasperReport-Vorlagen eingefügt werden können.


## Bedienung

1. HTML-Code kann eingegeben werden:
   - als direkter HTML-String (in Textfeld)
   - oder per Dateipfad aus `data/original/`
2. Konvertierungsoptionen einstellen:
   - Skalierungsfaktoren für X und Y
   - Elementtypen auswählen (Text, Tabellen, Bilder)
3. Konvertierung per Button starten
4. JasperReport-XML wird angezeigt und kann kopiert werden
5. Optional: XML in Datei speichern


## Technische Konstanten

- HTML_WIDTH = 1210
- HTML_HEIGHT = 825
- JASPER_PAGE_WIDTH = 595
- JASPER_PAGE_HEIGHT = 842
- JASPER_MARGIN_TOP = 20
- JASPER_MARGIN_RIGHT = 20
- JASPER_MARGIN_BOTTOM = 20
- JASPER_MARGIN_LEFT = 20


In [ ]:
# Benötigte Bibliotheken importieren
import sys
import os
import datetime
import re
import uuid

# Pfad zum shared-Verzeichnis hinzufügen, falls es nicht im Pythonpath ist
sys.path.append('..')

# Importieren der gemeinsam genutzten Funktionen
from shared.html_utils import (
    parse_html, 
    extract_css_styles, 
    extract_elements,
    convert_bottom_to_top,
    load_html_from_file, 
    save_original_html
)
from shared.constants import (
    HTML_HEIGHT, 
    HTML_WIDTH,
    JASPER_PAGE_WIDTH,
    JASPER_PAGE_HEIGHT,
    JASPER_MARGIN_TOP,
    JASPER_MARGIN_RIGHT,
    JASPER_MARGIN_BOTTOM,
    JASPER_MARGIN_LEFT,
    SCALE_FACTOR_X,
    SCALE_FACTOR_Y
)


In [ ]:
# Installieren von benötigten Paketen, falls nicht vorhanden
try:
    import bs4
    print(f"BeautifulSoup Version: {bs4.__version__}")
except ImportError:
    print("BeautifulSoup ist nicht installiert. Installation wird gestartet...")
    !pip install beautifulsoup4
    import bs4
    print(f"BeautifulSoup Version: {bs4.__version__}")

try:
    import ipywidgets as widgets
    from IPython.display import display, HTML
    print(f"ipywidgets Version: {widgets.__version__}")
except ImportError:
    print("ipywidgets ist nicht installiert. Installation wird gestartet...")
    !pip install ipywidgets
    import ipywidgets as widgets
    from IPython.display import display, HTML
    print(f"ipywidgets Version: {widgets.__version__}")


In [ ]:
# Funktionen für die JasperReport-XML-Generierung
def create_jasper_xml_header(page_width, page_height, margin_top, margin_right, margin_bottom, margin_left):
    """
    Erstellt den XML-Header für ein JasperReports-Dokument.
    
    Args:
        page_width (int): Seitenbreite
        page_height (int): Seitenhöhe
        margin_top (int): Oberer Rand
        margin_right (int): Rechter Rand
        margin_bottom (int): Unterer Rand
        margin_left (int): Linker Rand
        
    Returns:
        str: Der XML-Header
    """
    header = f"""<?xml version="1.0" encoding="UTF-8"?>
<!-- Created with HTML5 to JasperReports Converter -->
<jasperReport xmlns="http://jasperreports.sourceforge.net/jasperreports" 
              xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" 
              xsi:schemaLocation="http://jasperreports.sourceforge.net/jasperreports http://jasperreports.sourceforge.net/xsd/jasperreport.xsd" 
              name="HTML5_Converted_Report" 
              pageWidth="{page_width}" 
              pageHeight="{page_height}" 
              topMargin="{margin_top}" 
              rightMargin="{margin_right}" 
              bottomMargin="{margin_bottom}" 
              leftMargin="{margin_left}">
    <property name="com.jaspersoft.studio.data.defaultdataadapter" value="One Empty Record"/>
    <queryString>
        <![CDATA[]]>
    </queryString>
    <background>
        <band splitType="Stretch"/>
    </background>
    <title>
        <band height="{page_height - margin_top - margin_bottom}" splitType="Stretch">
"""
    return header

def create_jasper_xml_footer():
    """
    Erstellt den XML-Footer für ein JasperReports-Dokument.
    
    Returns:
        str: Der XML-Footer
    """
    footer = """        </band>
    </title>
</jasperReport>
"""
    return footer

def convert_text_element_to_jasper(element_id, element_text, style_dict, scale_factor_x, scale_factor_y):
    """
    Konvertiert ein HTML-Textelement in ein JasperReports-Element.
    
    Args:
        element_id (str): Die ID des Elements
        element_text (str): Der Text des Elements
        style_dict (dict): Die Stile des Elements
        scale_factor_x (float): Skalierungsfaktor für die X-Achse
        scale_factor_y (float): Skalierungsfaktor für die Y-Achse
        
    Returns:
        str: Das JasperReports-XML für das Element
    """
    # Standardwerte
    left = 0
    top = 0
    width = 100
    height = 20
    font_size = 10
    font_name = "Arial"
    is_bold = False
    is_italic = False
    
    # Positionierung aus Stilen extrahieren
    if 'left' in style_dict:
        match = re.search(r'(\d+\.?\d*)px', style_dict['left'])
        if match:
            left = float(match.group(1)) * scale_factor_x
    
    # Top-Position extrahieren oder aus Bottom berechnen
    if 'top' in style_dict:
        match = re.search(r'(\d+\.?\d*)px', style_dict['top'])
        if match:
            top = float(match.group(1)) * scale_factor_y
    elif 'bottom' in style_dict:
        match = re.search(r'(\d+\.?\d*)px', style_dict['bottom'])
        if match:
            bottom_px = float(match.group(1))
            top = (HTML_HEIGHT - bottom_px) * scale_factor_y
    
    # Schriftgröße extrahieren
    if 'font-size' in style_dict:
        match = re.search(r'(\d+\.?\d*)px', style_dict['font-size'])
        if match:
            font_size = float(match.group(1)) * min(scale_factor_x, scale_factor_y)
    
    # Schriftart extrahieren
    if 'font-family' in style_dict:
        font_name = style_dict['font-family'].split(',')[0].strip().replace("'", "").replace('"', '')
    
    # Fettdruck und Kursiv extrahieren
    if 'font-weight' in style_dict and style_dict['font-weight'] in ['bold', '700', '800', '900']:
        is_bold = True
    if 'font-style' in style_dict and style_dict['font-style'] == 'italic':
        is_italic = True
    
    # Eindeutige UUID für das Element generieren
    element_uuid = str(uuid.uuid4())
    
    # JasperReports-Textelement erstellen
    jasper_element = f"""            <staticText>
                <reportElement x="{int(left)}" y="{int(top)}" width="{int(width)}" height="{int(height)}" uuid="{element_uuid}"/>
                <textElement>
                    <font fontName="{font_name}" size="{int(font_size)}" isBold="{str(is_bold).lower()}" isItalic="{str(is_italic).lower()}"/>
                </textElement>
                <text><![CDATA[{element_text}]]></text>
            </staticText>
"""
    
    return jasper_element

def convert_html_to_jasper_snippets(html_string, scale_factor_x=SCALE_FACTOR_X, scale_factor_y=SCALE_FACTOR_Y, 
                                   include_header=True, include_footer=True, convert_bottom=True):
    """
    Konvertiert HTML-Code in JasperReports-XML-Snippets.
    
    Args:
        html_string (str): Der zu konvertierende HTML-Code
        scale_factor_x (float): Skalierungsfaktor für die X-Achse
        scale_factor_y (float): Skalierungsfaktor für die Y-Achse
        include_header (bool): Ob der XML-Header inkludiert werden soll
        include_footer (bool): Ob der XML-Footer inkludiert werden soll
        convert_bottom (bool): Ob bottom-Positionen zu top konvertiert werden sollen
        
    Returns:
        str: Das JasperReports-XML
    """
    # Wenn bottom zu top konvertiert werden soll
    if convert_bottom:
        html_string = convert_bottom_to_top(html_string)
    
    # HTML parsen
    soup = parse_html(html_string)
    if not soup:
        return "Fehler beim Parsen des HTML-Codes."
    
    # CSS-Stile extrahieren
    css_styles = extract_css_styles(soup)
    
    # JasperReports-XML erstellen
    jasper_xml = ""
    
    # Header hinzufügen, wenn gewünscht
    if include_header:
        jasper_xml += create_jasper_xml_header(
            JASPER_PAGE_WIDTH, JASPER_PAGE_HEIGHT,
            JASPER_MARGIN_TOP, JASPER_MARGIN_RIGHT,
            JASPER_MARGIN_BOTTOM, JASPER_MARGIN_LEFT
        )
    
    # Textelemente konvertieren
    for element_type in ['div', 'p', 'span']:
        elements = soup.find_all(element_type)
        for element in elements:
            element_id = element.get('id', '')
            if element_id and element_id in css_styles:
                element_text = element.get_text()
                jasper_xml += convert_text_element_to_jasper(
                    element_id, element_text, css_styles[element_id], 
                    scale_factor_x, scale_factor_y
                )
    
    # Footer hinzufügen, wenn gewünscht
    if include_footer:
        jasper_xml += create_jasper_xml_footer()
    
    return jasper_xml


In [ ]:
# Beispiel-HTML für die Demonstration
example_html = """<!DOCTYPE html>
<html>
<head>
    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
    <meta charset="utf-8" />
</head>

<body style="margin: 0;">

<div id="p1" style="overflow: hidden; position: relative; background-color: white; width: 1210px; height: 825px;">

    <!-- Begin shared CSS values -->
    <style class="shared-css" type="text/css" >
        .t {
            transform-origin: bottom left;
            z-index: 2;
            position: absolute;
            white-space: pre;
            overflow: visible;
            line-height: 1.5;
        }
        .text-container {
            white-space: pre;
        }
        @supports (-webkit-touch-callout: none) {
            .text-container {
                white-space: normal;
            }
        }
    </style>
    <!-- End shared CSS values -->

    <!-- Begin inline CSS -->
    <style type="text/css" >
        #t1_1{left:18px;top:21px;letter-spacing:0.14px;font-size:14px;font-family:'Arial';}
        #t2_1{left:128px;top:21px;font-size:14px;font-family:'Arial';font-weight:bold;}
        #t3_1{left:165px;top:21px;letter-spacing:0.15px;font-size:14px;font-family:'Arial';font-style:italic;}
        #t4_1{left:18px;bottom:804px;letter-spacing:0.14px;font-size:14px;font-family:'Arial';}
        #t5_1{left:128px;bottom:804px;font-size:14px;font-family:'Arial';font-weight:bold;}
        #t6_1{left:165px;bottom:804px;letter-spacing:0.15px;font-size:14px;font-family:'Arial';font-style:italic;}
    </style>
    <!-- End inline CSS -->

    <!-- Begin text -->
    <div id="t1_1" class="t">Beispieltext 1 (top)</div>
    <div id="t2_1" class="t">Beispieltext 2 (top, bold)</div>
    <div id="t3_1" class="t">Beispieltext 3 (top, italic)</div>
    <div id="t4_1" class="t">Beispieltext 4 (bottom)</div>
    <div id="t5_1" class="t">Beispieltext 5 (bottom, bold)</div>
    <div id="t6_1" class="t">Beispieltext 6 (bottom, italic)</div>
    <!-- End text -->

</div>
</body>
</html>"""


In [ ]:
# Erstellen der Widgets für die Benutzeroberfläche
html_input = widgets.Textarea(
    value=example_html,
    placeholder='HTML-Code hier einfügen',
    description='HTML-Code:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='200px')
)

file_input = widgets.Text(
    value='',
    placeholder='Dateipfad eingeben (z.B. original_2025-07-16_104156.html)',
    description='Datei:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

scale_factor_x = widgets.FloatSlider(
    value=SCALE_FACTOR_X,
    min=0.1,
    max=2.0,
    step=0.01,
    description='Skalierung X:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

scale_factor_y = widgets.FloatSlider(
    value=SCALE_FACTOR_Y,
    min=0.1,
    max=2.0,
    step=0.01,
    description='Skalierung Y:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

include_header = widgets.Checkbox(
    value=True,
    description='XML-Header inkludieren',
    disabled=False
)

include_footer = widgets.Checkbox(
    value=True,
    description='XML-Footer inkludieren',
    disabled=False
)

convert_bottom = widgets.Checkbox(
    value=True,
    description='Bottom zu Top konvertieren',
    disabled=False
)

convert_button = widgets.Button(
    description='Zu JasperReport konvertieren',
    disabled=False,
    button_style='success',
    tooltip='Klicken, um HTML zu JasperReport-XML zu konvertieren',
    icon='check'
)

save_button = widgets.Button(
    description='XML speichern',
    disabled=True,
    button_style='info',
    tooltip='Klicken, um XML in Datei zu speichern',
    icon='save'
)

output = widgets.Output()

# Anzeigen der Widgets
print("HTML-Code eingeben oder Datei auswählen:")
display(html_input)
print("\nODER Datei aus data/original/ auswählen:")
display(file_input)
print("\nKonvertierungsoptionen:")
display(scale_factor_x)
display(scale_factor_y)
display(widgets.HBox([include_header, include_footer, convert_bottom]))
display(convert_button)
display(save_button)
display(output)


In [ ]:
# Globale Variable für das generierte XML
generated_xml = ""

# Funktion für den Konvertierungsbutton
def on_convert_button_clicked(b):
    global generated_xml
    
    with output:
        output.clear_output()
        
        # HTML-Code aus Textfeld oder Datei laden
        html_string = ""
        if file_input.value:
            file_path = os.path.join('..', 'data', 'original', file_input.value)
            print(f"Lade HTML aus Datei: {file_path}")
            html_string = load_html_from_file(file_path)
            if not html_string:
                print("Fehler: Datei konnte nicht geladen werden.")
                return
        else:
            html_string = html_input.value
            if not html_string.strip():
                print("Fehler: Kein HTML-Code eingegeben.")
                return
            
            # Original-HTML speichern
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
            original_path = save_original_html(html_string, timestamp)
            print(f"Original-HTML gespeichert unter: {original_path}")
        
        # Konvertierungsoptionen auslesen
        scale_x = scale_factor_x.value
        scale_y = scale_factor_y.value
        inc_header = include_header.value
        inc_footer = include_footer.value
        conv_bottom = convert_bottom.value
        
        print(f"Konvertiere mit Skalierung X: {scale_x:.2f}, Y: {scale_y:.2f}")
        print(f"Header: {inc_header}, Footer: {inc_footer}, Bottom zu Top: {conv_bottom}")
        
        # HTML zu JasperReport konvertieren
        generated_xml = convert_html_to_jasper_snippets(
            html_string, 
            scale_factor_x=scale_x,
            scale_factor_y=scale_y,
            include_header=inc_header,
            include_footer=inc_footer,
            convert_bottom=conv_bottom
        )
        
        # Speicher-Button aktivieren
        save_button.disabled = False
        
        # XML anzeigen
        print("\nGeneriertes JasperReport-XML:")
        print(generated_xml)
        
        # Hinweis zur Verwendung
        print("\nHinweis: Das generierte XML kann in JasperReport-Vorlagen eingefügt werden.")
        print("Verwenden Sie den 'XML speichern'-Button, um das XML in eine Datei zu speichern.")

# Funktion für den Speicher-Button
def on_save_button_clicked(b):
    global generated_xml
    
    with output:
        if not generated_xml:
            print("Kein XML zum Speichern vorhanden.")
            return
        
        # XML-Datei erstellen
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
        xml_filename = f"jasper_report_{timestamp}.jrxml"
        xml_path = os.path.join('..', 'data', 'output', xml_filename)
        
        # Sicherstellen, dass der Ausgabeordner existiert
        os.makedirs(os.path.dirname(xml_path), exist_ok=True)
        
        # XML speichern
        with open(xml_path, 'w', encoding='utf-8') as file:
            file.write(generated_xml)
        
        print(f"JasperReport-XML wurde gespeichert unter: {xml_path}")

# Button-Klick-Events registrieren
convert_button.on_click(on_convert_button_clicked)
save_button.on_click(on_save_button_clicked)


## Erklärung der HTML zu JasperReport-Konvertierung

Die Konvertierung von HTML zu JasperReport-XML umfasst folgende Schritte:

1. **Parsen des HTML-Codes**: Extrahieren der Struktur und Stile
2. **Konvertieren von bottom zu top** (optional): Umrechnung der Positionierung
3. **Skalierung der Positionen und Größen**: Anpassung an JasperReport-Dimensionen
4. **Generierung von XML-Elementen**: Erstellung von JasperReport-kompatiblen XML-Snippets

Die wichtigsten Aspekte der Konvertierung sind:

- **Positionierung**: HTML verwendet Pixel, JasperReport verwendet Punkte (1/72 Zoll)
- **Skalierung**: Anpassung der Größenverhältnisse zwischen HTML und JasperReport
- **Textelemente**: Konvertierung von div, span, p zu staticText-Elementen
- **Schriftarten und Stile**: Übertragung von Schriftart, -größe, Fett- und Kursivdruck

Das generierte XML kann in JasperReport-Vorlagen eingefügt oder als eigenständige Vorlage verwendet werden.